In [1]:
import os
import numpy as np
import pandas as pd
from itertools import product

import matplotlib.pyplot as plt 



from scipy.signal import hilbert
from scipy.signal import butter, lfilter

from skimage.morphology import square, dilation, disk
from skimage.feature import canny
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,denoise_wavelet, estimate_sigma)
from skimage.filters import median
from skimage.io import imread, imsave
from skimage.color import rgb2gray
import cv2


from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression, BayesianRidge, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

from catboost import CatBoostRegressor, CatBoostClassifier
import lightgbm as lgb

#from Networks import *
#from Comp2_func import *
#from IOU import *

#from datetime import datetime
import gc
import warnings
warnings.filterwarnings("ignore")

In [2]:
def im_load():
    img_path= 'train_data/train_images/'
    msk_path= 'train_data/train_masks/'
    imx_bank= []
    imy_bank= []
    depths= []
    depth_data= pd.read_csv('depths.csv')
    for root, dirs, files in os.walk(img_path + '.'):  
        for file_name in files:
            imx= imread(img_path + file_name)
            imy= imread(msk_path + file_name)
            depth= depth_data['z'][depth_data['id']==file_name.split('.')[0]]

            imx= rgb2gray(imx)
            imy= imy / 65535
            if imx.max()>0:
                
                #imx= (imx - np.mean(imx)) / np.std(imx)
                size= (63, 63)
                imx= cv2.resize(imx, size)
                imy= cv2.resize(imy, size)
                
                imx_bank.append(imx)
                imy_bank.append(imy)
                depths.append(depth.values * np.ones((63*63, )).astype(int))  

    imx_bank= np.array(imx_bank)
    imy_bank= np.array(imy_bank)
    depths= np.array(depths)
    return imx_bank, imy_bank, depths

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def BP_filter(data, lowcut, highcut, fs, order, axis):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    if axis=='vertical':
        data= np.rot90(data)
    y = lfilter(b, a, data)
    if axis== 'vertical':
        y= np.rot90(y, k=1, axes=(1,0))
    return y

def BP_filter90(data, lowcut, highcut, fs, order):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    #data= np.rot90(data)
    y = lfilter(b, a, data)
    #y= np.rot90(y, k=1, axes=(1,0))
    return y

def bp(X, lowcut, highcut, order, axis):
    imx_bank= []
    for imx in X:
        bp= BP_filter(imx, lowcut=lowcut, highcut=highcut, fs=1/0.004, order=order, axis=axis)
        imx_bank.append(bp/1.3)
    imx_bank= np.array(imx_bank)
    return imx_bank


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def binary_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + (1 - dice_coef(y_true, y_pred))

def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef1(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = np.sum(np.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (np.sum(np.square(y_true),-1) + np.sum(np.square(y_pred),-1) + smooth)


# Show images
def show_im(X, y, imp):
    #imy= np.squeeze(y)
    #imx_show= np.squeeze(X)
    #imp= np.squeeze(imp, axis=[0, -1])

    fig, (ax1, ax2, ax3)= plt.subplots(1, 3)
    ax1.imshow(X)
    ax2.imshow(y)
    ax3.imshow(imp)
    plt.show()

In [3]:
print('Loading images')
X, y, d= im_load()
print(X.shape)

Loading images
(3920, 63, 63)


# Features

## Non-liniar

In [4]:
X2= X**2
Xlog= np.log(1 + X)

## Frequency BandPass

In [5]:
# Vertical axis
variations= []
low= [0.1, 2, 5, 10, 20] 
high= [7, 10, 20, 30, 40]
order= [2, 3, 5]
variations_prod= list(product(*[low, high, order]))
[variations.append(x) for x in variations_prod if x[1]>x[0]]

XBV= np.zeros((X.shape[0] * X.shape[1] * X.shape[2], len(variations)))
for i, variation in enumerate(variations): 
    XBV[:, i]= bp(X, lowcut=variation[0], highcut=variation[1], order=variation[2], axis='vertical').flatten() #reshape((X.shape[0], X.shape[1] * X.shape[2] ))

In [6]:
# Horizontal axis
variations= []
low= [0.1, 2, 5, 10, 20] 
high= [7, 10, 20, 30, 40]
order= [2, 3, 5]
variations_prod= list(product(*[low, high, order]))
[variations.append(x) for x in variations_prod if x[1]>x[0]]

XBH= np.zeros((X.shape[0] * X.shape[1] * X.shape[2], len(variations)))
for i, variation in enumerate(variations): 
    XBH[:, i]= bp(X, lowcut=variation[0], highcut=variation[1], order=variation[2], axis='horizontal').flatten()
  

## Analitic Signal

In [7]:
fs= 1/0.004
analytic_signal = hilbert(X)

amplitude_envelope = np.abs(analytic_signal)
envelope_derv1= np.insert((np.diff(amplitude_envelope)), 0, 0, axis=-1)
envelope_derv2= np.insert((np.diff(envelope_derv1)),0, 0, axis=-1)
instantaneous_phase = np.unwrap(np.angle(analytic_signal))
instantaneous_frequency = np.insert((np.diff(instantaneous_phase) / (2.0*np.pi) * fs),0, 0, axis=-1)

In [ ]:
for n in np.arange(0, len(X), 1):
    print(n)
    show_im(X[n], envelope_derv1[n], envelope_derv2[n])
    show_im(amplitude_envelope[n], instantaneous_phase[n], instantaneous_frequency[n])
    show_im(y[n], X2[n], Xlog[n])

# Flattening

In [8]:

X_f= np.expand_dims(X.flatten(), axis=1)
X2_f= np.expand_dims(X2.flatten(), axis=1)
Xlog_f= np.expand_dims(Xlog.flatten(), axis=1)
d_f= np.expand_dims(d.flatten(), axis=1)
ae_f=  np.expand_dims(amplitude_envelope.flatten(), axis=1)
aed1_f=  np.expand_dims(envelope_derv1.flatten(), axis=1)
aed2_f=  np.expand_dims(envelope_derv2.flatten(), axis=1)
ip_f=  np.expand_dims(instantaneous_phase.flatten(), axis=1)
if_f=  np.expand_dims(instantaneous_frequency.flatten(), axis=1)
y_f= (y.flatten()).astype(np.int8)


## PCA
* Need to merge test data for analysis.

In [9]:
pca1 = PCA()
Xpc= pca1.fit_transform(np.hstack((X_f, X2_f, Xlog_f, ae_f, aed1_f, aed2_f, ip_f, if_f)))
pca1.explained_variance_ratio_ 

array([9.96838581e-01, 1.75138007e-03, 1.22037141e-03, 1.45372145e-04,
       1.97878556e-05, 1.44052465e-05, 1.00932509e-05, 8.57259470e-09])

In [ ]:
Xpc1= Xpc[:, 0].reshape(-1, 63, 63)
Xpc2= Xpc[:, 1].reshape(-1, 63, 63)
Xpc3= Xpc[:, 2].reshape(-1, 63, 63)
Xpc4= Xpc[:, 3].reshape(-1, 63, 63)
Xpc5= Xpc[:, 4].reshape(-1, 63, 63)
Xpc6= Xpc[:, 5].reshape(-1, 63, 63)
Xpc7= Xpc[:, 6].reshape(-1, 63, 63)

In [ ]:
n=15
show_im(X[n], Xpc1[n], Xpc2[n])
show_im(Xpc3[n], Xpc4[n], Xpc5[n])
show_im(Xpc6[n], Xpc7[n], y[n])



In [10]:
pca2 = PCA()
XBVpc= pca2.fit_transform(XBV)
pca2.explained_variance_ratio_ 

array([3.64064524e-01, 2.40532070e-01, 1.64632955e-01, 9.59964170e-02,
       4.73215540e-02, 3.42361093e-02, 1.91960203e-02, 1.31947027e-02,
       8.42757475e-03, 4.59525830e-03, 4.10732153e-03, 2.23285485e-03,
       8.38500946e-04, 3.27341237e-04, 1.91655353e-04, 7.40147537e-05,
       2.01859405e-05, 7.14954709e-06, 2.58553819e-06, 6.23720955e-07,
       4.91784993e-07, 4.43825111e-08, 3.76721871e-08, 6.34564780e-09,
       6.00475524e-10, 4.47604766e-10, 9.58191042e-12, 4.09385192e-12,
       4.66921237e-13, 1.72512401e-13, 8.12914215e-14, 1.85251789e-14,
       4.74595546e-15, 3.80577121e-15, 2.04739655e-15, 1.50878398e-15,
       5.37691318e-17, 3.85673004e-17, 2.13692108e-17, 3.35165451e-18,
       9.41897011e-19, 4.11390882e-19, 2.53507034e-19, 1.56826383e-19,
       1.03711440e-19, 1.82185621e-20, 6.00652884e-21, 4.34323899e-21,
       1.82294074e-21, 1.65292595e-22, 1.47528436e-22, 7.56926201e-23,
       3.29703785e-23, 3.91658517e-24, 1.29968356e-24, 6.97815071e-25,
      

In [ ]:
BPVpc1= XBVpc[:, 0].reshape(-1, 63, 63)
BPVpc2= XBVpc[:, 1].reshape(-1, 63, 63)
BPVpc3= XBVpc[:, 2].reshape(-1, 63, 63)
BPVpc4= XBVpc[:, 3].reshape(-1, 63, 63)
BPVpc5= XBVpc[:, 4].reshape(-1, 63, 63)
BPVpc6= XBVpc[:, 5].reshape(-1, 63, 63)
BPVpc7= XBVpc[:, 6].reshape(-1, 63, 63)

In [ ]:
n=15
show_im(X[n], BPVpc1[n], BPVpc2[n])
show_im(BPVpc3[n], BPVpc4[n], BPVpc5[n])
show_im(BPVpc6[n], BPVpc7[n], y[n])


In [11]:
pca3 = PCA()
XBHpc= pca3.fit_transform(XBH)
pca3.explained_variance_ratio_ 

array([3.71556074e-01, 2.74417926e-01, 1.48545107e-01, 8.61069293e-02,
       4.47241329e-02, 2.80412702e-02, 1.83580507e-02, 1.13102744e-02,
       6.46218315e-03, 3.96266621e-03, 3.49337644e-03, 1.82044285e-03,
       6.45422972e-04, 3.13963961e-04, 1.55251922e-04, 6.15203861e-05,
       1.61481228e-05, 5.91883100e-06, 2.33154876e-06, 5.05696676e-07,
       4.24145371e-07, 3.86237040e-08, 3.32192985e-08, 5.91143918e-09,
       5.56718916e-10, 3.87860278e-10, 8.40377556e-12, 3.86328252e-12,
       4.27199718e-13, 1.78875224e-13, 8.40970854e-14, 1.72620012e-14,
       4.91323973e-15, 3.58174477e-15, 2.06428622e-15, 1.56772558e-15,
       5.17241887e-17, 3.81996220e-17, 2.17399882e-17, 3.17245332e-18,
       9.82043565e-19, 4.19881360e-19, 2.53946479e-19, 1.51041132e-19,
       1.04645147e-19, 1.76667840e-20, 6.19361393e-21, 4.20081030e-21,
       1.87765773e-21, 1.65154351e-22, 1.45442398e-22, 7.76917673e-23,
       3.40342114e-23, 3.81183161e-24, 1.28804153e-24, 6.99574299e-25,
      

## Median Encoding

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
enc = KBinsDiscretizer(n_bins=100, encode='ordinal')
X_binned = enc.fit_transform(Xpc[:, 0].reshape(-1, 1))

In [ ]:
for q in range(100):
    #z= X_binned[X_binned==q]
    z1= y_f.reshape(-1,1)[X_binned==q]
    plt.hist(z1)

# TSNE

In [ ]:
from sklearn.manifold import TSNE
tsne5= TSNE(n_components=3, perplexity= 5, learning_rate= 300, 
           n_iter= 250, n_iter_without_progress= 5, 
           random_state= 0, verbose= 5)
tsne60= TSNE(n_components=3, perplexity= 60, learning_rate= 300, 
           n_iter= 1000, n_iter_without_progress= 5, 
           random_state= 0, verbose= 5)
tsne100= TSNE(n_components=3, perplexity= 100, learning_rate= 300, 
           n_iter= 250, n_iter_without_progress= 5, 
           random_state= 0, verbose= 5)


#data= np.hstack((d_f[4*63*63:7*63*63, :], Xpc[4*63*63:7*63*63, :]))

Xsne5= tsne5.fit_transform(data)
#Xsne60= tsne60.fit_transform(data)
#Xsne100= tsne100.fit_transform(data)

In [ ]:
plt.figure(figsize=(15, 15))
plt.scatter(Xsne5[:, 1], Xsne5[:, 2], s= 0.7, c=y_f[4*63*63:7*63*63]);plt.show()

In [ ]:
plt.figure(figsize=(15, 15))
plt.scatter(Xsne100[:, 0], Xsne100[:, 1], s= 0.7, c=y_f[4*63*63:7*63*63]);plt.show()

In [ ]:
plt.figure(figsize=(15, 15))
plt.scatter(Xsne30[:, 1], Xsne30[:, 2], s= 0.7, c=y_f[4*63*63:7*63*63]);plt.show()

## KNN features

In [48]:
data= np.hstack((d_f, Xpc))
scaler = MinMaxScaler()
X_modeling= scaler.fit_transform(data)

knn_model= KNeighborsRegressor(n_neighbors=20, n_jobs=12)

#ynn= y_f[4*63*63:7*63*63]
knn_model.fit(data, y_f)
dist, _= knn_model.kneighbors(data)
dist= dist[:, 1:]
dist.shape

(15558480, 19)

In [49]:
Stas= []
for w in [3, 9, 19]:
    Stas.append(dist[:, :w].mean(axis=1))
    Stas.append(dist[:, :w].std(axis=1))
Stas= np.array(Stas).T
Stas.shape

(15558480, 6)

## Data

In [12]:
#X_stacked= np.hstack((X_f, X2_f, Xlog_f, d_f, XBV, XBH, ae_f, aed1_f, aed2_f, ip_f, if_f))

X_modeling= np.hstack((d_f, Xpc, XBVpc, XBHpc))#, Stas

scaler = MinMaxScaler()
X_modeling= scaler.fit_transform(X_modeling)
#print(X_modeling.max(), X_modeling.shape)

Xpc= scaler.fit_transform(Xpc)
XBVpc= scaler.fit_transform(XBVpc)
XBHpc= scaler.fit_transform(XBHpc)
#Stas= scaler.fit_transform(Stas)

# Modeling

## Level 1

In [ ]:
level1= []

lgb_params= {'feature_fraction': 0.7,'metric': 'rmse', 'nthread':12, 'min_data_in_leaf': 2**7, 
                  'bagging_fraction': 0.7, 'learning_rate': 0.05, 'objective': 'rmse',
                  'bagging_seed': 2**5, 'num_leaves': 2**11,'bagging_freq':1}

for level1_X in [X_modeling[:, :61], X_modeling[:, :21], Xpc, XBVpc, XBHpc]:#, Stas
    
    #print('CAT')
    #level1_CATmodel = CatBoostRegressor(iterations=200, learning_rate=0.05, 
     #                                   min_data_in_leaf= 2**7, depth=5, 
      #                                  random_seed= 0, verbose= 50)
    #level1_CATmodel.fit(level1_X, y_f)
    #level1_pCAT= level1_CATmodel.predict(level1_X)
    #level1.append(level1_pCAT)

    print('LGB')
    level1_LGBmodel = lgb.train(lgb_params, lgb.Dataset(level1_X, label=y_f), 50)
    p1= level1_LGBmodel.predict(level1_X)
    error= y_f-p + 1
    
    print('LGB_error')
    lgb_params= {'feature_fraction': 0.7,'metric': 'rmse', 'nthread':12, 'min_data_in_leaf': 2**7, 
                  'bagging_fraction': 0.7, 'learning_rate': 0.05, 'objective': 'rmse',
                  'bagging_seed': 2**5, 'num_leaves': 2**11,'bagging_freq':1,'verbosity': 50}
    
    level1_LGBmodel = lgb.train(lgb_params, lgb.Dataset(level1_X, label=y_f, weight=error), 300)
    p= level1_LGBmodel.predict(level1_X)
    error1= y_f-p1
    
    print('LGB_error1')
    level1_LGBmodel = lgb.train(lgb_params, lgb.Dataset(level1_X, label=error1), 30)
    p2= level1_LGBmodel.predict(level1_X)
    
    level1.append(p1 + p2)
    ax = lgb.plot_importance(level1_LGBmodel, figsize=(12, 30));plt.show()
    
    #print('BR')
    #level1_X= np.hstack((level1_X, np.ones((level1_X.shape[0], 1))))
    #level1_BRmodel= BayesianRidge()
    #level1_BRmodel.fit(level1_X, y_f)
    #level1_pBR= level1_BRmodel.predict(level1_X)
    #level1.append(level1_pBR)
    
level1= np.array(level1).T 
level1.max(axis=0)    

In [17]:
#y_f.shape
error.shape

(15558480,)

In [ ]:
level= level1
for col in range(level.shape[1]):
    dice= []
    thresholds= np.arange(0.1, level[:, col].max(), 0.01)
    for threshold in thresholds:
        p_test= np.zeros((level.shape[0], 1))

        p_test[level[:, col] > threshold]= 1
        p_test= p_test.astype(np.float32)
        y_true= y_f.astype(np.float32)

        dice.append(dice_coef1(y_true, p_test.squeeze()))
    plt.plot(thresholds, dice)
    max_dice= np.array(dice).max()
    best_th= thresholds[np.argmax(np.array(dice))]
    print('max dice is ' + str(max_dice) + ' at threshold ' + str(best_th))

## Level 2

In [22]:
X_level2= level1
X_level2.shape
scaler1 = MinMaxScaler()
X_level2= scaler1.fit_transform(X_level2)

In [ ]:
level2= []

print('BR')
level2_BRmodel= BayesianRidge()
level2_BRmodel.fit(X_level2, y_f)
level2_pBR= level2_BRmodel.predict(X_level2)
print(level2_pBR.max(), level2_BRmodel.coef_)
level2.append(level2_pBR)

#print('SVR')
#level2_SVRmodel= SVR(kernel= 'linear', gamma='scale', C=1.0, epsilon=0.2, max_iter= 300)
#level2_SVRmodel.fit(X_level2, y_f)
#level2_pSVR= level2_SVRmodel.predict(X_level2)
#level2.append(level2_pSVR)

print('KNN')
level2_KNNmodel= KNeighborsRegressor(n_neighbors=3, n_jobs=12)
level2_KNNmodel.fit(X_level2, y_f)
level2_pKNN= level2_KNNmodel.predict(X_level2)
level2.append(level2_pKNN)

print('LGB')
level2_LGBmodel = lgb.train(lgb_params, lgb.Dataset(X_level2, label=y_f), 500)
level2_pLGB= level2_LGBmodel.predict(X_level2)
ax = lgb.plot_importance(level2_LGBmodel, figsize=(12, 30));plt.show()


#print('Ada')
#level2_Adamodel = AdaBoostRegressor(DecisionTreeRegressor(max_depth= 3, 
 #                                                         min_samples_leaf= 2**7), 
  #                                  learning_rate=0.1, 
   #                                 random_state=0, 
    #                                n_estimators=2)

#level2_Adamodel.fit(level2_X, y_f)
#level2_pAda= level2_Adamodel.predict(level2_X)
#level2.append(level2_pAda) 

#print('Ada')
#level2_Adamodel = AdaBoostRegressor(base_estimator= GradientBoostingRegressor(subsample= 0.7, 
 #                                                                             min_samples_leaf= 2**7, 
  #                                                                            max_depth=5), 
   #                                 learning_rate=0.1, 
    #                                random_state=0, 
     #                               n_estimators=2)

#level2_Adamodel.fit(level2_X, y_f)
#level2_pAda= level2_Adamodel.predict(level2_X)
#level2.append(level2_pAda) 




level2.append(level2_pLGB)
level2= np.array(level2).T 
level2.max(axis=0)    

In [ ]:
level= level2
for col in range(level.shape[1]): 
    dice= []
    thresholds= np.arange(0.1, level[:, col].max(), 0.01)
    for threshold in thresholds:
        p_test= np.zeros((level.shape[0], 1))

        p_test[level[:, col] > threshold]= 1
        p_test= p_test.astype(np.float32)
        y_true= y_f.astype(np.float32)

        dice.append(dice_coef1(y_true, p_test.squeeze()))
    plt.plot(thresholds, dice)
    max_dice= np.array(dice).max()
    best_th= thresholds[np.argmax(np.array(dice))]
    print('max dice is ' + str(max_dice) + 'at threshold ' + str(best_th))

## Level 3

In [25]:
scaler2 = MinMaxScaler()
X_level3= scaler2.fit_transform(level2)

In [26]:
level3_BRmodel= BayesianRidge()
level3_BRmodel.fit(X_level3, y_f)
level3_pBR= level3_BRmodel.predict(X_level3)
print(level3_pBR.max(), level3_BRmodel.coef_)

1.1788653189180875 [-0.40211719  0.95485541  0.39739217]


In [ ]:
print('SVR')
level3_SVRmodel= SVR(kernel= 'liniar', gamma='scale', C=1.0, epsilon=0.2, max_iter= 1000)
level3_SVRmodel.fit(X_level3, y_f)
level3_pSVR= level3_SVRmodel.predict(X_level3)
#level3.append(level3_pSVR)

In [ ]:
print('KNN')
level3_KNNmodel= KNeighborsRegressor(n_neighbors=3, n_jobs=12)
level3_KNNmodel.fit(X_level3, y_f)
level3_pKNN= level3_KNNmodel.predict(X_level3)
level3.append(level3_pKNN)

In [ ]:
level= level3_pBR

dice= []
thresholds= np.arange(0.1, level.max(), 0.01)
for threshold in thresholds:
    p_test= np.zeros((level.shape))

    p_test[level > threshold]= 1
    p_test= p_test.astype(np.float32)
    y_true= y_f.astype(np.float32)

    dice.append(dice_coef1(y_true, p_test.squeeze()))
plt.plot(thresholds, dice)
max_dice= np.array(dice).max()
best_th= thresholds[np.argmax(np.array(dice))]
print('max dice is ' + str(max_dice) + 'at threshold ' + str(best_th))

In [28]:
p_final= np.zeros((level3_pBR.shape))
p_final[level3_pBR > best_th]= 1

In [ ]:
imp= p_final.reshape((-1, 63, 63))
y_c= y_true.reshape((-1, 63, 63))

for i in np.arange(0, len(X), 5):
    print(i)
    show_im(X[i], y_c[i], imp[i])

# Irosion/Dilation/AutoEncoder